In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
import moscot
from anndata import AnnData
import numpy as np
from moscot.backends.ott import FGWSolver, SinkhornSolver, GWSolver, LRSinkhornSolver
from moscot.solvers._data import TaggedArray, Tag
import matplotlib.pyplot as plt
import jax.numpy as jnp
from typing import *
import pandas as pd
import networkx as nx
import jax.numpy as jnp
from moscot.problems.time._lineage import TemporalProblem
import os
import wot

In [9]:
data_path = "/Users/DominikKlein/Documents/PhD/data/wot"


VAR_GENE_DS_PATH = os.path.join(data_path, 'ExprMatrix.var.genes.h5ad')
CELL_DAYS_PATH = os.path.join(data_path, 'cell_days.txt')
SERUM_CELL_IDS_PATH = os.path.join(data_path, 'serum_cell_ids.txt')
CELL_GROWTH_PATH = os.path.join(data_path, 'growth_gs_init.txt')

In [10]:
adata = wot.io.read_dataset(VAR_GENE_DS_PATH, obs=[CELL_DAYS_PATH, CELL_GROWTH_PATH], obs_filter=SERUM_CELL_IDS_PATH)
adata.shape

(175472, 1479)

In [ ]:
adata.X = adata.X.todense()

In [11]:
ot_model = wot.ot.OTModel(adata,epsilon = 0.05, lambda1 = 1,lambda2 = 50) 

In [12]:
tmap_annotated = ot_model.compute_transport_map(7,7.5)

In [13]:
tmap_annotated.obs

,g0,g1
index,,
D7_Dox_C1_AAACCTGAGGAATCGC-1,1.268895,0.977585
D7_Dox_C1_AAACCTGAGTAGATGT-1,2.978660,2.510212
D7_Dox_C1_AAACCTGCAACCGCCA-1,1.871918,1.789105
D7_Dox_C1_AAACCTGCACTATCTT-1,3.072989,2.790364
D7_Dox_C1_AAACCTGCAGCCTGTG-1,1.026225,0.918578
...,...,...
D7_Dox_C2_TTTGTCAAGTTAGCGG-1,0.338531,0.370486
D7_Dox_C2_TTTGTCACAGTGGAGT-1,0.582584,0.609163
D7_Dox_C2_TTTGTCAGTAATTGGA-1,0.254926,0.296703


In [27]:
tp = TemporalProblem(adata, solver=SinkhornSolver(jit=False))

In [28]:
adata.obs.day = adata.obs.day.astype('category')

In [29]:
tp.prepare(key="day", subset=[7, 7.5])

In [ ]:
tp.solve(tau_a=0.5, tau_b=0.5)

In [ ]:
ot_model.tmap.shape, tp.